In [ ]:
import uproot
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import awkward as ak
import selection
from helpers import *
%load_ext autoreload

In [ ]:
branches = [
    "slc.vertex.x",
    "slc.vertex.y",
    "slc.vertex.z",
    "slc.truth.index",
    "slc.tmatch.pur",
    "slc.tmatch.eff",
    "slc.truth.iscc",
    "slc.truth.pdg",
    "slc.truth.E",
    "slc.truth.position.x",
    "slc.truth.position.y",
    "slc.truth.position.z",
    "slc.self",
    "slc.nu_score",
    "slc.fmatch.score",
    "slc.fmatch.time",
    "slc.fmatch_a.score",
    "slc.is_clear_cosmic",
    "reco.ntrk",
    "reco.nshw",
    
    "slc.reco.trk.len",
    "slc.reco.trk.costh",
    "slc.reco.trk.phi",
    "slc.reco.trk.start.x",
    "slc.reco.trk.start.y",
    "slc.reco.trk.start.z",
    "slc.reco.trk.end.x",
    "slc.reco.trk.end.y",
    "slc.reco.trk.end.z",
    "slc.reco.trk.ID",
    "slc.reco.trk.slcID",
    "slc.reco.trk.parent_is_primary",
    "slc.reco.trk.chi2pid0.chi2_muon",
    "slc.reco.trk.chi2pid0.chi2_proton",
    "slc.reco.trk.chi2pid1.chi2_muon",
    "slc.reco.trk.chi2pid1.chi2_proton",
    "slc.reco.trk.chi2pid2.chi2_muon",
    "slc.reco.trk.chi2pid2.chi2_proton",
    "slc.reco.trk.bestplane",
    "slc.reco.trk.mcsP.fwdP_muon",
    "slc.reco.trk.rangeP.p_muon",
    "slc.reco.trk.rangeP.p_proton",
    "slc.reco.trk.crthit.hit.time",
    "slc.reco.trk.crthit.distance",
    "slc.reco.trk.crttrack.angle",
    "slc.reco.trk.crttrack.time",
    "slc.reco.trk.truth.bestmatch.G4ID",
    "slc.reco.trk.truth.bestmatch.energy",
    "slc.reco.trk.truth.p.pdg",
    "slc.reco.trk.truth.p.planeVisE",
    "slc.reco.trk.truth.p.gen.x",
    "slc.reco.trk.truth.p.gen.y",
    "slc.reco.trk.truth.p.gen.z",
    "slc.reco.trk.truth.p.genp.x",
    "slc.reco.trk.truth.p.genp.y",
    "slc.reco.trk.truth.p.genp.z",
    "slc.reco.trk.truth.p.end.x",
    "slc.reco.trk.truth.p.end.y",
    "slc.reco.trk.truth.p.end.z",
    "slc.reco.trk.truth.p.length",
    "slc.reco.trk.truth.p.contained",
    "slc.reco.trk.truth.p.end_process",
    "slc.reco.trk.truth.total_deposited_energy",
    "mc.nnu",
    "mc.nu.E",
    "mc.nu.pdg",
    "mc.nu.iscc",
    
    "true_particles.G4ID",
    "true_particles.pdg",
    
    "pass_flashtrig",
    "crt_hits.time",
    
    "nslc",
    "slc.reco.ntrk",
]

In [ ]:
treenames = [
    "rec",
    "rec.slc",
    "rec.slc.reco.trk",
    "rec.true_particles",
    "rec.crt_hits",
    "rec.mc.nu",
    "rec.reco.trk",
    "rec.reco.trk.chi2pid",
    "rec.reco.trk.truth.matches",
]

In [ ]:
do_save = True
savedir = "./plots/"

In [ ]:
fname = "sbnd-overlay.flat.root"
rootf = uproot.open(fname)

data = {}
for b in branches:
    keyname = "rec." + b # ".".join(b.split(".")[1:])
    for t in treenames:
        if not keyname.startswith(t):
            continue
        try:
            d = rootf["recTree"][t].array(keyname)
        except KeyError:
            continue
        data[b] = d
        break
    else:
        raise KeyError(keyname)

groupings = ["slc.reco.trk"]
for k in data.keys():
    for g in groupings:
        if k.startswith(g):
            data[k] = group(data[k], data[g.replace(g.split(".")[-1], "n"+g.split(".")[-1])])

to_broadcast = ["pass_flashtrig"]
broadcast_over = "nslc"
for k in to_broadcast:
    data[k] = broadcast(data[k], data[broadcast_over])

In [ ]:
is_nu = data["slc.truth.index"] >= 0
is_cosmic = np.invert(is_nu)
is_numu_cc = is_nu & data["slc.truth.iscc"] & (np.abs(data["slc.truth.pdg"]) == 14)

def dist(x1, y1, z1, x2, y2, z2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)

is_fid_true = selection.InFV(data["slc.truth.position.x"], data["slc.truth.position.y"], data["slc.truth.position.z"])
is_fid_reco = selection.InFV(data["slc.vertex.x"], data["slc.vertex.y"], data["slc.vertex.z"])

data["slc.reco.trk.truth.p.inAV"] = selection.InAV(data["slc.reco.trk.truth.p.gen.x"], data["slc.reco.trk.truth.p.gen.y"], data["slc.reco.trk.truth.p.gen.z"]) &\
            selection.InFV(data["slc.reco.trk.truth.p.end.x"], data["slc.reco.trk.truth.p.end.y"], data["slc.reco.trk.truth.p.end.z"])
data["slc.reco.trk.contained"] = selection.InAV(data["slc.reco.trk.end.x"], data["slc.reco.trk.end.y"], data["slc.reco.trk.end.z"])
data["slc.reco.trk.atslc"] = dist(data["slc.reco.trk.start.x"], data["slc.reco.trk.start.y"], data["slc.reco.trk.start.z"],
                                 data["slc.vertex.x"], data["slc.vertex.y"], data["slc.vertex.z"]) < 10

data["slc.reco.trk.truth.p.is_stopping"] = (data["slc.reco.trk.truth.p.end_process"] == 1) |\
                                    (data["slc.reco.trk.truth.p.end_process"] == 2) |\
                                    (data["slc.reco.trk.truth.p.end_process"] == 3) |\
                                    (data["slc.reco.trk.truth.p.end_process"] == 41)

data["slc.reco.trk.recop"] = data["slc.reco.trk.rangeP.p_muon"] + 0. # clone
data["slc.reco.trk.recop"][np.invert(data["slc.reco.trk.contained"])] = data["slc.reco.trk.mcsP.fwdP_muon"][np.invert(data["slc.reco.trk.contained"])] 

for chi2 in ["chi2_muon", "chi2_proton"]:
    data["slc.reco.trk.bestplane." + chi2] = data["slc.reco.trk.chi2pid2." + chi2]
    data["slc.reco.trk.bestplane." + chi2][data["slc.reco.trk.bestplane"] == 0] = data["slc.reco.trk.chi2pid0." + chi2][data["slc.reco.trk.bestplane"] == 0]
    data["slc.reco.trk.bestplane." + chi2][data["slc.reco.trk.bestplane"] == 1] = data["slc.reco.trk.chi2pid1." + chi2][data["slc.reco.trk.bestplane"] == 1]

# For the cases where two slices match to the same neutrino, figure out which match is "primary"
# Use whichever slice gets more of the deposited energy
data["slc.truth.match_is_primary"] = data["slc.truth.iscc"] & False # clone
# Check the max match for each neutrino match 
# Group each slice by spill
index_spill = group(data["slc.truth.index"], data["nslc"])
eff_spill = group(data["slc.tmatch.eff"], data["nslc"])
primary_spill = group(data["slc.truth.match_is_primary"], data["nslc"])

for i in range(data["slc.truth.index"].max()+1): # consider up to the maximum nuetrinos
    primary_spill = primary_spill | (eff_spill[index_spill==i].max() == eff_spill) & (eff_spill[index_spill==i].max() > 0.)

data["slc.truth.match_is_primary"] = primary_spill.flatten()
# all cosmic matches are primary
data["slc.truth.match_is_primary"] = data["slc.truth.match_is_primary"] | (data["slc.truth.index"] < 0)

In [ ]:
# Verify that the truth matching worked as expected

# bad 
for i in range(data["slc.truth.index"].max()+1):
    bad = ((index_spill == i) & primary_spill).sum() > 1
    assert(not bad.any())
    
# also bad
for i in range(data["slc.truth.index"].max()+1):
    bad = ((index_spill == i).sum() > 0) & (np.invert(data["slc.truth.match_is_primary"].any()))
    assert(not bad.any())

when = ((primary_spill == False) & (index_spill >= 0)).any()

#print(eff_spill[when][0])
#print(index_spill[when][0])
#print(primary_spill[when][0])

In [ ]:
is_muon = np.abs(data["slc.reco.trk.truth.p.pdg"]) == 13
is_proton = np.abs(data["slc.reco.trk.truth.p.pdg"]) == 2212
var = data["slc.reco.trk.bestplane.chi2_muon"]
pure = (data["slc.reco.trk.truth.bestmatch.energy"] / data["slc.reco.trk.truth.total_deposited_energy"] > 0.5)
complete = (data["slc.reco.trk.truth.bestmatch.energy"] / data["slc.reco.trk.truth.p.planeVisE"] > 0.5)
when = pure & complete & data["slc.truth.match_is_primary"]

_ = plt.hist([var[is_numu_cc & is_muon & when].flatten(), var[is_numu_cc & is_proton & when].flatten()],
        label=["CC $\\nu_\\mu$ $\\mu$", "CC $\\nu_\\mu$ $p$"],
        bins = np.linspace(0, 50, 26),
        histtype="step")
_ = plt.legend()
plt.xlabel(r'$\chi^2_\mathrm{muon}$')
plt.ylabel("Unnormalized Entries")

if do_save: plt.savefig(savedir + "chi2_muon.png")

In [ ]:
is_muon = np.abs(data["slc.reco.trk.truth.p.pdg"]) == 13
is_proton = np.abs(data["slc.reco.trk.truth.p.pdg"]) == 2212
is_contained = data["slc.reco.trk.truth.p.contained"]
is_stopping = data["slc.reco.trk.truth.p.is_stopping"]

var = data["slc.reco.trk.bestplane.chi2_muon"]
pure = (data["slc.reco.trk.truth.bestmatch.energy"] / data["slc.reco.trk.truth.total_deposited_energy"] > 0.5)
complete = (data["slc.reco.trk.truth.bestmatch.energy"] / data["slc.reco.trk.truth.p.planeVisE"] > 0.5)
when = pure & complete & data["slc.truth.match_is_primary"]

_ = plt.hist([var[is_numu_cc & is_muon & is_contained & when].flatten(), 
              var[is_numu_cc & is_muon & np.invert(is_contained) & when].flatten(),
              var[is_numu_cc & is_proton & is_contained & is_stopping& when].flatten(),
              var[is_numu_cc & is_proton & is_contained & np.invert(is_stopping) & when].flatten(),
              var[is_numu_cc & is_proton & np.invert(is_contained) & when].flatten()],
        label=["CC $\\nu_\\mu$ $\\mu$ (Contained)", "CC $\\nu_\\mu$ $\\mu$ (Exiting)", 
               "CC $\\nu_\\mu$ $p$ (Stopping)", "CC $\\nu_\\mu$ $p$ (Scattering)", "CC $\\nu_\\mu$ $p$ (Exiting)"],
        bins = np.linspace(0, 50, 26),
        histtype="step")
_ = plt.legend(loc="upper center")
plt.xlabel(r'$\chi^2_\mathrm{muon}$')
plt.ylabel("Unnormalized Entries")

if do_save: plt.savefig(savedir + "chi2_muon_detail.png")

In [ ]:
var = data["slc.reco.trk.bestplane.chi2_proton"]

_ = plt.hist([var[is_numu_cc & is_muon & when].flatten(), var[is_numu_cc & is_proton & when].flatten()],
        label=["CC $\\nu_\\mu$ $\\mu$", "CC $\\nu_\\mu$ $p$"],
        bins = np.linspace(0, 300, 31),
        histtype="step")
_ = plt.legend()

plt.xlabel(r'$\chi^2_\mathrm{proton}$')
plt.ylabel("Unnormalized Entries")

if do_save: plt.savefig(savedir + "chi2_proton.png")

In [ ]:
primary_track = selection.get_primary_tracks(data)
primary_track_ind = ak.JaggedArray.fromcounts((primary_track >= 0)*1, primary_track[primary_track >= 0])
true_primary_track = selection.get_true_primary_track(data)
true_primary_track_ind = ak.JaggedArray.fromcounts((true_primary_track >= 0)*1, true_primary_track[true_primary_track >= 0])

In [ ]:
has_true_primary_track = true_primary_track >= 0

valid = (data["slc.reco.trk.truth.p.length"] > 50.) &\
        (data["slc.reco.trk.truth.p.inAV"] | (data["slc.reco.trk.truth.p.length"] > 100.)) &\
        is_numu_cc & is_fid_true

valid_primary = primary_track == np.nan # copy
valid_primary[primary_track_ind.count() > 0] = valid[primary_track_ind].flatten()

In [ ]:
var = data["slc.truth.E"]
good_ptrack = primary_track[valid_primary & data["slc.truth.match_is_primary"]] == true_primary_track[valid_primary & data["slc.truth.match_is_primary"]]
acc = np.sum(good_ptrack) / len(good_ptrack)
_ = plt.hist([var[valid_primary & data["slc.truth.match_is_primary"]], var[valid_primary & data["slc.truth.match_is_primary"]][good_ptrack]],
             label=["All $\\nu_\\mu$ CC", "Correct Muon $\\nu_\\mu$ CC"],
             histtype="step",
             bins=np.linspace(0, 3, 21))
plt.text(0.75, 0.5, 'Total Accuracy: %.1f%%' % (acc*100), horizontalalignment='center',fontsize=14, transform=plt.gca().transAxes)
plt.legend()
plt.xlabel("$\\nu$ Energy [GeV]")
plt.ylabel("Unnormalized Entries")

if do_save: plt.savefig(savedir + "primary_track_acc.png")

In [ ]:
# Set all of the primary track slice variables
keys = list(data.keys())
for k in keys:
    if k.startswith("slc.reco.trk."):
        slc_key = k.replace(".reco.trk.", ".ptrk.")
        data[slc_key] = np.empty(data["slc.nu_score"].shape)
        data[slc_key][:] = np.NaN
        
        is_bool = data[k][0].dtype == "bool"
        d = data[k] + 0 # clone
        data[slc_key][primary_track >= 0] = d[primary_track_ind[primary_track_ind >= 0]].flatten()
        if is_bool:
            data[slc_key] = data[slc_key] == 1 # restore bool-type arrays
data["slc.has_ptrk"] = primary_track >= 0

In [ ]:
var = data["slc.ptrk.len"]
when = valid_primary
_ = plt.hist([var[when & (np.abs(data["slc.ptrk.truth.p.pdg"]) == 211)],
             var[when & (np.abs(data["slc.ptrk.truth.p.pdg"]) == 2212)]],
             label=["Pion", "Proton"],
             bins=np.linspace(50, 400, 21),
            histtype="step")
plt.xlabel("Reconstructed Track Length [cm]")
plt.ylabel("Unnormalized Entries")
plt.text(0.75, 0.5, "Length of Mistaken\n Primary Tracks\n by Particle Type", horizontalalignment='center',fontsize=14, transform=plt.gca().transAxes)

plt.legend()

if do_save: plt.savefig(savedir + "primary_track_wrong_len.png")

In [ ]:
# Define the Cuts!!!!!
fid = selection.fid(data)
nu_score = selection.nu_score(data)
ptrk = selection.ptrk(data)

In [ ]:
# look at the momentum reconstruction of CCnumu muons that have the necessary lengths

varx = np.sqrt(data["slc.ptrk.truth.p.genp.x"]**2 + data["slc.ptrk.truth.p.genp.y"]**2 + data["slc.ptrk.truth.p.genp.z"]**2)
vary = data["slc.ptrk.recop"]

when = is_numu_cc & is_fid_true & fid & ptrk & data["slc.ptrk.contained"]
binx = np.linspace(0.2, 1, 21)
biny = np.linspace(0.2, 1, 21)
h,_,_,_ = plt.hist2d(varx[when], vary[when],
              bins=[binx,biny])
_ = plt.plot([0.2, 1], [0.2, 1], color="r")
plt.xlabel("True Primary Track Momentum [GeV]")
plt.ylabel("Reco Primary Track Momentum [GeV]")
c = plt.colorbar()
c.set_label("Unnormalized Entries")
plt.text(0.35, 0.8, 'Contained Tracks\n Range Momentum Reco', color="w", horizontalalignment='center',fontsize=14, transform=plt.gca().transAxes)

vmax = np.max(h)
if do_save: plt.savefig(savedir + "range_momentum.png")

In [ ]:
# look at the momentum reconstruction of CCnumu muons that have the necessary lengths

varx = np.sqrt(data["slc.ptrk.truth.p.genp.x"]**2 + data["slc.ptrk.truth.p.genp.y"]**2 + data["slc.ptrk.truth.p.genp.z"]**2)
vary = data["slc.ptrk.recop"]

when = is_numu_cc & is_fid_true & fid & ptrk & np.invert(data["slc.ptrk.contained"])
binx = np.linspace(0.2, 1, 21)
biny = np.linspace(0.2, 1, 21)
h,_,_,_ = plt.hist2d(varx[when], vary[when], #vmax=vmax,
              bins=[binx,biny])
_ = plt.plot([0.2, 1], [0.2, 1], color="r")
plt.xlabel("True Primary Track Momentum [GeV]")
plt.ylabel("Reco Primary Track Momentum [GeV]")
c = plt.colorbar()
c.set_label("Unnormalized Entries")

plt.text(0.3, 0.8, 'Exiting Tracks\n MCS Momentum Reco', color="w", horizontalalignment='center',fontsize=14, transform=plt.gca().transAxes)

vmax = np.max(h)
if do_save: plt.savefig(savedir + "mcs_momentum.png")